## Introduction to Spark

In [12]:
%python
//!wget https://sadatashareagsparkml.blob.core.windows.net/hadoop-bangalore/flight-retail-data.zip -O ~/data/flight-retail-data.zip

IndexError: ('pop from empty list',)

In [13]:
%python
!unzip -n ~/data/flight-retail-data.zip -d ~/data/

Archive:  /home/atingupta2005/data/flight-retail-data.zip



IndexError: ('pop from empty list',)

In [14]:
%python
!ls ~/data/flight-retail-data/flight-data/csv/2015-summary.csv

/home/atingupta2005/data/flight-retail-data/flight-data/csv/2015-summary.csv



IndexError: ('pop from empty list',)

In [19]:
// in Scala
val flightData2015 = spark
  .read
  .option("inferSchema", "true")
  .option("header", "true")
  .csv("../../data/flight-retail-data/flight-data/csv/2015-summary.csv")

flightData2015: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [20]:
flightData2015.take(3)

res5: Array[org.apache.spark.sql.Row] = Array([United States,Romania,15], [United States,Croatia,1], [United States,Ireland,344])


In [21]:
flightData2015.sort("count").explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#40 ASC NULLS FIRST], true, 0
   +- Exchange rangepartitioning(count#40 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [id=#52]
      +- FileScan csv [DEST_COUNTRY_NAME#38,ORIGIN_COUNTRY_NAME#39,count#40] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/atingupta2005/data/flight-retail-data/flight-data/csv/2015-..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [22]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [23]:
flightData2015.sort("count").take(2)

res8: Array[org.apache.spark.sql.Row] = Array([United States,Singapore,1], [Moldova,United States,1])


In [24]:
flightData2015.createOrReplaceTempView("flight_data_2015")

In [25]:
// in Scala
val sqlWay = spark.sql("""
SELECT DEST_COUNTRY_NAME, count(1)
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
""")


sqlWay: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, count(1): bigint]


In [26]:
val dataFrameWay = flightData2015
  .groupBy('DEST_COUNTRY_NAME)
  .count()

sqlWay.explain
dataFrameWay.explain

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#38, 5), ENSURE_REQUIREMENTS, [id=#74]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#38] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/atingupta2005/data/flight-retail-data/flight-data/csv/2015-..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[count(1)])
   +- Exchange hashpartitioning(DEST_COUNTRY_NAME#38, 5), ENSURE_REQUIREMENTS, [id=#87]
      +- HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[partial_count(1)])
         +- FileScan csv [DEST_COUNTRY_NAME#38] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIn

dataFrameWay: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, count: bigint]


In [27]:
spark.sql("SELECT max(count) from flight_data_2015").take(1)

res11: Array[org.apache.spark.sql.Row] = Array([370002])


In [28]:
// in Scala
import org.apache.spark.sql.functions.max


import org.apache.spark.sql.functions.max


In [29]:
flightData2015.select(max("count")).show()

+----------+
|max(count)|
+----------+
|    370002|
+----------+



In [30]:
// in Scala
val maxSql = spark.sql("""
SELECT DEST_COUNTRY_NAME, sum(count) as destination_total
FROM flight_data_2015
GROUP BY DEST_COUNTRY_NAME
ORDER BY sum(count) DESC
LIMIT 5
""")

maxSql.show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



maxSql: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, destination_total: bigint]


In [31]:
// in Scala
import org.apache.spark.sql.functions.desc

flightData2015
  .groupBy("DEST_COUNTRY_NAME")
  .sum("count")
  .withColumnRenamed("sum(count)", "destination_total")
  .sort(desc("destination_total"))
  .limit(5)
  .show()

+-----------------+-----------------+
|DEST_COUNTRY_NAME|destination_total|
+-----------------+-----------------+
|    United States|           411352|
|           Canada|             8399|
|           Mexico|             7140|
|   United Kingdom|             2025|
|            Japan|             1548|
+-----------------+-----------------+



import org.apache.spark.sql.functions.desc


In [32]:
// in Scala
flightData2015
  .groupBy("DEST_COUNTRY_NAME")
  .sum("count")
  .withColumnRenamed("sum(count)", "destination_total")
  .sort(desc("destination_total"))
  .limit(5)
  .explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- TakeOrderedAndProject(limit=5, orderBy=[destination_total#148L DESC NULLS LAST], output=[DEST_COUNTRY_NAME#38,destination_total#148L])
   +- HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[sum(count#40)])
      +- Exchange hashpartitioning(DEST_COUNTRY_NAME#38, 5), ENSURE_REQUIREMENTS, [id=#287]
         +- HashAggregate(keys=[DEST_COUNTRY_NAME#38], functions=[partial_sum(count#40)])
            +- FileScan csv [DEST_COUNTRY_NAME#38,count#40] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/atingupta2005/data/flight-retail-data/flight-data/csv/2015-..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,count:int>




In [33]:
// in Scala
import spark.implicits._
case class Flight(DEST_COUNTRY_NAME: String,
                  ORIGIN_COUNTRY_NAME: String,
                  count: BigInt)
val flightsDF = spark.read
  .parquet("/home/atingupta2005/data/flight-retail-data/flight-data/parquet/2010-summary.parquet/")

// Convert to dataset
val flights = flightsDF.as[Flight]

import spark.implicits._
defined class Flight
flightsDF: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]
flights: org.apache.spark.sql.Dataset[Flight] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [34]:
flightsDF

res16: org.apache.spark.sql.DataFrame = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [35]:
flights.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [36]:
flights
  .filter(flight_row => flight_row.ORIGIN_COUNTRY_NAME != "Canada")
  .map(fr => Flight(fr.DEST_COUNTRY_NAME, fr.ORIGIN_COUNTRY_NAME, fr.count + 5))

res18: org.apache.spark.sql.Dataset[Flight] = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


In [37]:
flights.show()

+--------------------+-------------------+-----+
|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+--------------------+-------------------+-----+
|       United States|            Romania|    1|
|       United States|            Ireland|  264|
|       United States|              India|   69|
|               Egypt|      United States|   24|
|   Equatorial Guinea|      United States|    1|
|       United States|          Singapore|   25|
|       United States|            Grenada|   54|
|          Costa Rica|      United States|  477|
|             Senegal|      United States|   29|
|       United States|   Marshall Islands|   44|
|              Guyana|      United States|   17|
|       United States|       Sint Maarten|   53|
|               Malta|      United States|    1|
|             Bolivia|      United States|   46|
|            Anguilla|      United States|   21|
|Turks and Caicos ...|      United States|  136|
|       United States|        Afghanistan|    2|
|Saint Vincent and..

In [38]:
// in Scala
val staticDataFrame = spark.read.format("csv")
  .option("header", "true")
  .option("inferSchema", "true")
  .load("../data/retail-data/by-day/*.csv")

staticDataFrame.createOrReplaceTempView("retail_data")
val staticSchema = staticDataFrame.schema

org.apache.spark.sql.AnalysisException:  Path does not exist: file:/home/atingupta2005/my_code/data/retail-data/by-day/*.csv

In [39]:
spark.sql("""
select * from retail_data where Country = 'United Kingdom'
""").show()

org.apache.spark.sql.AnalysisException:  Table or view not found: retail_data; line 2 pos 14;

In [53]:
// in Scala
import org.apache.spark.sql.functions.{window, column, desc, col}
staticDataFrame
  .selectExpr(
    "CustomerId",
    "(UnitPrice * Quantity) as total_cost",
    "InvoiceDate")
  .groupBy(
    col("CustomerId"), window(col("InvoiceDate"), "1 day"))
  .sum("total_cost")
  .show(5)

+----------+--------------------+------------------+
|CustomerId|              window|   sum(total_cost)|
+----------+--------------------+------------------+
|   14075.0|{2011-12-05 00:00...|316.78000000000003|
|   18180.0|{2011-12-05 00:00...|            310.73|
|   15358.0|{2011-12-05 00:00...| 830.0600000000003|
|   15392.0|{2011-12-05 00:00...|304.40999999999997|
|   15290.0|{2011-12-05 00:00...|263.02000000000004|
+----------+--------------------+------------------+
only showing top 5 rows



import org.apache.spark.sql.functions.{window, column, desc, col}


In [40]:
spark.conf.set("spark.sql.shuffle.partitions", "5")